# Final Version of the GravRag Populator:

In [18]:
!pip install requests
!pip install markdown
!pip install beautifulsoup4

In [ ]:
"""
dsadsa

dsads
a
d
sa
"""

In [37]:
import os
import time
import logging
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
import requests
import markdown
from bs4 import BeautifulSoup

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# API endpoint information
baseUrl = "http://localhost:8000"

# Function to send request with metadata
def send_request_with_metadata(title, metadata):
    url = f"{baseUrl}/gravrag/create_memory"  # Use your existing API endpoint
    headers = {"Content-Type": "application/json"}
    body = {"content": title, "metadata": metadata}
    
    try:
        response = requests.post(
            url,
            headers=headers,
            data=json.dumps(body)  # Convert the body to JSON format
        )
        
        if response.status_code == 200:
            return response.json()  # Return the response as JSON
        else:
            logger.error(f"Request failed with status code {response.status_code}: {response.text}")
            return {"error": f"Request failed with status code {response.status_code}", "details": response.text}
    except requests.RequestException as e:
        logger.error(f"Request failed: {e}")
        return {"error": "Request failed", "details": str(e)}

# Data model to represent sections of the README
@dataclass
class ReadmeSection:
    content: str
    heading: str
    level: int
    parent: Optional[str]
    children: List[str]
    metadata: Dict[str, Any]

# Function to parse README and break it into sections
def parse_readme(content: str) -> List[ReadmeSection]:
    html = markdown.markdown(content)
    soup = BeautifulSoup(html, 'html.parser')
    sections = []
    section_stack = []
    current_section = None

    for elem in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'pre', 'ul', 'ol']):
        if elem.name.startswith('h'):
            level = int(elem.name[1])
            while section_stack and section_stack[-1].level >= level:
                section_stack.pop()

            current_section = ReadmeSection(
                content='',
                heading=elem.text.strip(),
                level=level,
                parent=section_stack[-1].heading if section_stack else None,
                children=[],
                metadata={}
            )

            # Append to parent only if there's a parent
            if section_stack:
                section_stack[-1].children.append(current_section.heading)

            sections.append(current_section)
            section_stack.append(current_section)
        else:
            if current_section:
                current_section.content += " " + elem.get_text(separator=" ", strip=True)


    #Strip any leading/trailing spaces in content for clean output
    for section in sections:
        section.content = section.content.strip()

    return sections


# Function to process README and send sections to API
def process_readme_and_send(readme_path: str):
    with open(readme_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    sections = parse_readme(content)
    
    for section in sections:
        # Prepare title and metadata without vectorization
        title = section.heading
        metadata = {
            "content": section.content,
            "level": section.level,
            "parent": section.parent,
            "children": section.children,
            "timestamp": time.time(),
            "access_count": 0,
            "relevance_score": 1.0
        }

        # Send to the API endpoint
        response = send_request_with_metadata(title, metadata)
        print(f"Sent section '{title}' to API. Metadata: {metadata}")
    

## TO RUN:

In [38]:
process_readme_and_send("README.MD") #THIS IS THE FILE PATH TO THE README

Sent section 'Table of Contents' to API. Metadata: {'content': '1. Overview 2. Backend Components 2.1. Schemas 2.1.1. User Schema 2.1.2. Project Schema 2.1.3. Task Schema 2.1.4. Submission Schema 2.1.5. Comment Schema 2.1.6. Notification Schema 2.1.7. Email Schema 2.1.8. Leaderboard Schema 2.2. Server Setup 2.2.1. Imports and Dependencies 2.2.2. Express Application Configuration 2.2.3. Middleware Configuration 2.2.4. File Upload Handling 2.2.5. Authentication Middleware 2.2.6. Request Logging 2.3. Utility Functions 2.3.1. createNewTask Function 2.4. Routes 2.4.1. Authentication Routes 2.4.2. Task Management Routes 2.4.3. Submission Management Routes 2.4.5. Comment Management Routes 2.4.6. Admin Routes 2.4.7. Miscellaneous Routes 3. Frontend Components 3.1. HTML Structure 3.2. CSS and Styling 3.3. JavaScript Functionality 4. Security Considerations 5. Deployment and Environment Configuration 6. Conclusion 2.1. Schemas 2.1.1. User Schema 2.1.2. Project Schema 2.1.3. Task Schema 2.1.4. Su

Sent section 'Schemas' to API. Metadata: {'content': 'The backend utilizes Mongoose to define and interact with MongoDB schemas. Each schema represents a distinct entity within the system, encapsulating relevant data and relationships.', 'level': 3, 'parent': 'Backend Components', 'children': [], 'timestamp': 1728320364.8360653, 'access_count': 0, 'relevance_score': 1.0}
Sent section 'User Schema' to API. Metadata: {'content': "Purpose : Represents a user within the Groqy system. Fields : username (String): User's display name. Defaults to an empty string. email (String): User's email address. Defaults to an empty string. password_hash (String): Hashed password for secure authentication. Defaults to an empty string. role (String): Defines user role, such as 'user' or 'admin'. Defaults to 'user'. created_at (Date): Timestamp of user creation. Defaults to the current date and time. updated_at (Date): Timestamp of the last profile update. Defaults to the current date and time. bio (String

Sent section 'Comment Schema' to API. Metadata: {'content': 'Purpose : Allows users to leave comments on tasks. Fields : user_id (String): ID of the user leaving the comment. Defaults to null . task_id (String): ID of the task being commented on. Defaults to null . content (String): Text content of the comment. Defaults to an empty string. created_at (Date): Timestamp of comment creation. Defaults to the current date and time. updated_at (Date): Timestamp of the last comment update. Defaults to the current date and time. user_id (String): ID of the user leaving the comment. Defaults to null . task_id (String): ID of the task being commented on. Defaults to null . content (String): Text content of the comment. Defaults to an empty string. created_at (Date): Timestamp of comment creation. Defaults to the current date and time. updated_at (Date): Timestamp of the last comment update. Defaults to the current date and time.', 'level': 3, 'parent': 'Backend Components', 'children': [], 'time

Sent section 'File Upload Handling' to API. Metadata: {'content': "Multer Configuration : Storage : Configured to store files in the 'uploads' directory with unique filenames to prevent conflicts. Filename Structure : Combines the original field name, a unique suffix (timestamp and random number), and the original file extension. File Size Limit : Set to 50MB to accommodate large files. Upload Middleware : upload is an instance of Multer configured with the defined storage and file size limits. Used in routes that handle file uploads, ensuring files are processed and stored correctly. Storage : Configured to store files in the 'uploads' directory with unique filenames to prevent conflicts. Filename Structure : Combines the original field name, a unique suffix (timestamp and random number), and the original file extension. File Size Limit : Set to 50MB to accommodate large files. upload is an instance of Multer configured with the defined storage and file size limits. Used in routes tha

Sent section 'Promote User to Admin' to API. Metadata: {'content': "Endpoint : POST /api/admin/promote Purpose : Allows an admin to elevate a user's role to 'admin'. Access Control : Restricted to users with the 'admin' role. Process : Checks if the authenticated user ( req.user ) has the 'admin' role. If not, responds with a 403 Forbidden . Extracts the email from the request body. Searches for the user with the provided email. If the user is not found, responds with a 404 Not Found . Updates the user's role to 'admin'. Saves the updated user to the database. Responds with a success message indicating the user has been promoted. Checks if the authenticated user ( req.user ) has the 'admin' role. If not, responds with a 403 Forbidden . Extracts the email from the request body. Searches for the user with the provided email. If the user is not found, responds with a 404 Not Found . Updates the user's role to 'admin'. Saves the updated user to the database. Responds with a success message

...

Sent section '26. users/route.js' to API. Metadata: {'content': "File Path in Next.js: ```bash\napp/api/users/route.js ``` Purpose: Manages user-related operations, including fetching user information, updating user details, and deleting users. This route is primarily intended for administrative tasks, allowing admins to perform bulk user management actions. Key Components/Functions: GET : Retrieves a list of all users with task summaries (admin only). POST , PUT , DELETE : Potentially handles user-related actions, though specific functionalities may be defined in sub-routes. Implementation Guidelines: Purpose : Facilitate comprehensive user management, enabling administrators to retrieve, update, and delete user data as required. Key Functions : get : Functionality : Fetches all users from the database along with summaries of their associated tasks. Authorization : Accessible only to users with the 'admin' role. post , put , delete : Potential Uses : Handle bulk user creation, updates

Sent section '28. users/me/route.js' to API. Metadata: {'content': "File Path in Next.js: ```bash\napp/api/users/me/route.js ``` Purpose: Provides an endpoint for authenticated users to retrieve their own profile information. This allows users to view and manage their personal data within the system. Key Components/Functions: GET : Retrieves the profile information of the authenticated user. PUT : Updates the profile information of the authenticated user. DELETE : Potentially allows users to delete their own accounts. Implementation Guidelines: Purpose : Enable users to access and manage their personal profile information securely. Key Functions : get : Functionality : Fetches the authenticated user's profile information from the database. Excludes sensitive information like password_hash . Authorization : Accessible only to authenticated users. put : Functionality : Allows users to update their profile details (e.g., username, bio, skills). Authorization : Ensures that users can only 

Sent section '30. users/[id]/route.js' to API. Metadata: {'content': "File Path in Next.js: ```bash\napp/api/users/[id]/route.js ``` Purpose: Handles operations for a specific user identified by their unique ID. This includes retrieving, updating, and deleting individual user accounts. Primarily intended for administrative actions, allowing admins to manage user accounts effectively. Key Components/Functions: GET : Retrieves details of a specific user. PUT : Updates the details of a specific user. DELETE : Deletes a specific user account. Implementation Guidelines: Purpose : Enable detailed management of individual user accounts, allowing for retrieval, modification, and removal based on the user's unique identifier. Key Functions : get : Functionality : Fetches the user identified by the [id] parameter. Includes user details and associated task summaries. Authorization : Accessible only to admins or the user themselves. put : Functionality : Updates user details such as username, emai

Sent section '31. Summary and Additional Considerations' to API. Metadata: {'content': "Having detailed the core API routes, it's essential to consider the following aspects to ensure a robust and maintainable Next.js 14 application:", 'level': 2, 'parent': None, 'children': ['1. Middleware and Authentication', '2. Database Integration', '3. File Uploads and Storage', '4. Frontend Components', '5. Security Best Practices', '6. Error Handling and Logging', '7. Performance Optimization', '8. Testing and Deployment', '9. Documentation', '10. Scalability Considerations'], 'timestamp': 1728320547.333091, 'access_count': 0, 'relevance_score': 1.0}
Sent section '1. Middleware and Authentication' to API. Metadata: {'content': "Middleware Implementation : Utilize Next.js's built-in middleware to handle authentication and authorization. Implement a global middleware to verify JWT tokens on protected routes. Role-Based Access Control (RBAC) : Define user roles (e.g., 'user', 'admin') and assign p

Sent section '10. Scalability Considerations' to API. Metadata: {'content': 'Horizontal Scaling : Design the application to support horizontal scaling, allowing it to handle increased load by adding more instances. Database Scaling : Implement database indexing and sharding strategies to manage large datasets efficiently. Load Balancing : Use load balancers to distribute incoming traffic evenly across server instances. Design the application to support horizontal scaling, allowing it to handle increased load by adding more instances. Implement database indexing and sharding strategies to manage large datasets efficiently. Use load balancers to distribute incoming traffic evenly across server instances.', 'level': 3, 'parent': '31. Summary and Additional Considerations', 'children': [], 'timestamp': 1728320568.1752205, 'access_count': 0, 'relevance_score': 1.0}
Sent section '32. Conclusion' to API. Metadata: {'content': "By following this comprehensive guide, you can effectively transit

# Function to Recall memories from the GravRAG using only TITLE:

- SCROLL DOWN FOR THE TEST RUN EXAMPLE

In [3]:
import requests
import json
baseUrl = "http://localhost:8000"
def recall_memories_using_title(title: str, top_k):
    url = f"{baseUrl}/gravrag/recall_memory"
    headers = {
        "Content-Type": "application/json"
    }
    body = {
        "query": title,
        "top_k": top_k
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(body))

        if response.status_code == 200:
            return response.json()  # Return the JSON response if successful
        else:
            return {
                "error": f"Request failed with status code {response.status_code}",
                "details": response.text
            }
    except requests.RequestException as e:
        return {
            "error": "Request failed",
            "details": str(e)
        }


## TEST RUN:

### TITLE - TITLE THAT YOU WANT TO SEARCH FOR: 

### TOP_K - NUMBER OF ITEMS YOU WANT TO RETRIEVE

In [4]:
title = "User Schema' to API. Metadata" # EXAMPLE TITLE
top_k = 5 #NUMBER OF ITEMS I WANT TO GET FROM DATABASE

print(recall_memories_using_title(title,top_k))

{'memories': [{'content': 'Previous Conversation between the User and the Model: [{"role":"user","content":"You are a an English teacher."},{"role":"user","content":"hello"}]', 'metadata': {'user': {'role': 'user', 'question': 'You are a an English teacher. hello'}, 'assistant': {'role': 'assistant', 'response': 'Hello there.  How can I assist you today? Are you in need of help with a specific English assignment, or do you have questions about grammar, vocabulary, or literature?'}, 'timestamp': 1728435555.5308692, 'recall_count': 0, 'memetic_similarity': 1.0, 'semantic_relativity': 0.4002832748706773, 'gravitational_pull': 0.4002832431585618, 'spacetime_coordinate': 0.0002629297362859214}}, {'content': 'User completed the onboarding task for Project X', 'metadata': {'timestamp': 1728436892.2732816, 'recall_count': 0, 'memetic_similarity': 1.0, 'semantic_relativity': 0.1497188116815651, 'gravitational_pull': 0.1497188019101073, 'spacetime_coordinate': 0.0008064363593215891}}]}


# Function to recall memories from the GravRAG using Title AND Metadata:

- SCROLL DOWN FOR THE TEST RUN EXAMPLE

### Please Note that the Metadata MUST be EXACTLY THE SAME as the one stored inside the DATABASE

In [41]:
import requests
import json

baseUrl = "http://localhost:8000"

def recall_memories_using_title_and_metadata(title, top_k, metadata):
    
    url = f"{baseUrl}/gravrag/recall_with_metadata"
    headers = {
        "Content-Type": "application/json"
    }
    
    # Construct the request body with query, metadata, and top_k
    body = {
        "query": title,
        "metadata": metadata,
        "top_k": top_k
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(body))

        if response.status_code == 200:
            return response.json()  # Return the JSON response if successful
        else:
            return {
                "error": f"Request failed with status code {response.status_code}",
                "details": response.text
            }
    except requests.RequestException as e:
        return {
            "error": "Request failed",
            "details": str(e)
        }


## TEST RUN:

### TITLE - TITLE THAT YOU WANT TO SEARCH FOR: 

### TOP_K - NUMBER OF ITEMS YOU WANT TO RETRIEVE

### METADATA - METADATA THAT YOU WANT TO SEARCH FOR

In [45]:
title = "Notification Schema" # EXAMPLE TITLE
top_k = 5 #NUMBER OF ITEMS I WANT TO GET FROM DATABASE
metadata = {
    "content": "Purpose : Stores notifications for users. Fields : user_id (String): ID of the user receiving the notification. Defaults to null . type (String): Type of notification, such as 'task_assigned' or 'submission_feedback'. Defaults to an empty string. content (String): Content/message of the notification. Defaults to an empty string. related_id (String): ID of the related entity (task, project, submission, etc.). Defaults to null . read (Boolean): Indicates if the notification has been read. Defaults to false . created_at (Date): Timestamp of notification creation. Defaults to the current date and time. read_at (Date): Timestamp of when the notification was read. Defaults to null . user_id (String): ID of the user receiving the notification. Defaults to null . type (String): Type of notification, such as 'task_assigned' or 'submission_feedback'. Defaults to an empty string. content (String): Content/message of the notification. Defaults to an empty string. related_id (String): ID of the related entity (task, project, submission, etc.). Defaults to null . read (Boolean): Indicates if the notification has been read. Defaults to false . created_at (Date): Timestamp of notification creation. Defaults to the current date and time. read_at (Date): Timestamp of when the notification was read. Defaults to null .",
    "level": 3,
    "parent": "Backend Components",
    "children": [],
    "access_count": 0,
    "relevance_score": 1,
    "recall_count": 0,
    "memetic_similarity": 1,
    "semantic_relativity": 1,
} #metadata example

recall_memories_using_title_and_metadata(title,top_k,metadata) 

{'memories': [{'content': 'Notification Schema',
   'metadata': {'content': "Purpose : Stores notifications for users. Fields : user_id (String): ID of the user receiving the notification. Defaults to null . type (String): Type of notification, such as 'task_assigned' or 'submission_feedback'. Defaults to an empty string. content (String): Content/message of the notification. Defaults to an empty string. related_id (String): ID of the related entity (task, project, submission, etc.). Defaults to null . read (Boolean): Indicates if the notification has been read. Defaults to false . created_at (Date): Timestamp of notification creation. Defaults to the current date and time. read_at (Date): Timestamp of when the notification was read. Defaults to null . user_id (String): ID of the user receiving the notification. Defaults to null . type (String): Type of notification, such as 'task_assigned' or 'submission_feedback'. Defaults to an empty string. content (String): Content/message of the 